In [2]:

import sys
import tensorflow as tf
import tensorflow_hub as hub
import pandas as pd
import numpy as np
import os
from os.path import isfile,join
import xml.etree.cElementTree as ET
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
import time
import re
import pickle
#import doc2vec_impl
import gensim
import nltk
nltk.download('stopwords')
nltk.download('punkt')

import FileProcess as fp


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


('Found embeddings: ', 0, '/', 12195)
('word_index len = ', 12195)
('time-taken for FP: ', 1.6829960346221924)


In [28]:
print (fp.window_size)
batch_size=10
vocabulary_size=400000
embedding_size=50
learning_rate = 0.1
#embedding_filename = "glove_6B_200d.txt"
embedding_filename = "word2vec.model"   ##Trying with pre-trained word2vec model
training_epochs=1
module_url = "https://tfhub.dev/google/universal-sentence-encoder/2" #@param ["https://tfhub.dev/google/universal-sentence-encoder/2", "https://tfhub.dev/google/universal-sentence-encoder-large/3"]


2


In [4]:
dirpath = "corpus/fulltext/"
files = [f for f in os.listdir(dirpath) if isfile(join(dirpath, f))]

print(fp.get_sentences(files[0]))

[' background  1 the applicant is a citizen of the peoples republic of china (prc).', 'he arrived in australia on 25 december 2002.', 'on 14 january 2003 the applicant applied for a protection (class xa) visa.', 'on 18 june 2003 a delegate of the minister for immigration and multicultural affairs (as he was then) refused to grant the applicant a protection (class xa) visa ("delegate\'s decision").', "on 7 july 2003 the applicant applied to the refugee review tribunal for review of the delegate's decision.", 'on 5 may 2004 the applicant gave oral evidence at a tribunal hearing.', "on 3 march 2005 the tribunal made a decision affirming the delegate's decision.", 'on 23 march 2006 federal magistrate lloyd-jones made orders by consent remitting the matter to the tribunal.', 'on 24 may 2006 the tribunal, as reconstituted, sent a letter, by facsimile, inviting the applicant to attend a hearing.', 'on 26 may 2006 the tribunal received a "response to hearing invitation" form indicating that th

In [5]:
stop_words_list=set(stopwords.words('english'))
tokenizer = RegexpTokenizer(r'\w+')

##########loading dev2vec=============
#doc2vec_impl.main();

d2v_model = gensim.models.doc2vec.Doc2Vec.load('doc2vec_models_50len/doc2vec.model')
docvec = d2v_model.docvecs[files[0]]
print(docvec.shape)

(50,)


In [6]:
#
def read_data(file_name):
    with open(file_name, 'r') as f:
        word_vocab = set()  # not using list to avoid duplicate entry
        word_count = {};
        word_index={};

        word2vector = {}
        count=0;
        for line in f:
            line_ = line.strip()  # Remove white space
            words_Vec = line_.split()
            word_vocab.add(words_Vec[0])
            word_index[words_Vec[0]]=count;
            count=count+1;

            word2vector[words_Vec[0]] = np.array(words_Vec[1:], dtype=float)
    print("Total Words in DataSet:", len(word_vocab))
    return word_vocab, word2vector, word_index

In [7]:
def load_embeddings(filename):
    dvocab, w2v,word_index  = read_data(filename);
    return w2v,word_index

In [54]:
embedding_array = fp.embedding_array
word_index = fp.word_index


reverseWordIndex = {}
for key in word_index:
    reverseWordIndex[word_index[key]] = key

In [ ]:
# ## reducing word embeddings to 50
# if embedding_size==50:
#     rand = np.random.rand(200,embedding_size)
#     embedding_array_50={} 
#     for key in embedding_array:
#         embedding_array_50[key] = np.matmul(embedding_array[key].reshape(1, 200), rand)
#     print (embedding_array_50['biennials'].shape)

In [9]:
#### TO avoid downloading tf hub modules everytime
##  export TFHUB_CACHE_DIR=/usr/local/bin

#sen2vec embeddings
def sen2vec1(sentences):
    embed = hub.Module(module_url)
    with tf.Session() as session:
        session.run([tf.global_variables_initializer(), tf.tables_initializer()])
        sentenceEmbeddings = session.run(embed(sentences))    
        return sentenceEmbeddings

    
def sent2vector(sent):
    words = word_tokenize(sent.lower())
    emb=[]
    for w in words:
        if w in embedding_array.keys():
            emb.append(embedding_array.get(w));
        else:
            emb.append(embedding_array.get('unk'));
    return np.mean(np.array(emb),axis=0)

def get_sent_embeddings(sent_list):
    out_vec=np.zeros((len(sent_list)-1,200))
    for i  in range(len(sent_list)-1):
        out_vec[i]=sent2vector(sent_list[i]);
    return out_vec
    
# sentence_embeddings = sen2Vec(['UWA was ordered to pay the costs of Dr Gra','Sirtex succeeded in its cross-claim against Dr'])

In [11]:
def get_statements(file):
    #with open(dirpath + file, 'r',encoding="utf-8", errors='replace') as f:
    with open(dirpath + file, 'r') as f:
        data=str(f.read());
        data=data.lower()
        data = data.replace("\"id=", "id=\"");
        data=data.replace("\n","")
        data=data.replace('".*?=.*?"', "",)
        data=data.replace("&","");
        xml = ET.fromstring(str(data))
        name=None;
        rows_list=[];
        catchphrases=[];
        sentences=[];
        for child in xml:
            if child.tag=="catchphrases":
                for catchphrase in child:
                    id=catchphrase.attrib.get("id")
                    #print(catchphrase.text)
                    catchphrases.append({"file_id":file,"Name":name,"Id":id,"text":catchphrase.text})
                    #catchphrases+=tokenizer.tokenize(catchphrase.text)
            if child.tag=="sentences":
                for sentence in child:
                    id = sentence.attrib.get("id")
                    sentences.append({"file_id":file,"Name":name,"Id":id,"text":sentence.text})
                    #sentences+=tokenizer.tokenize(sentence.text)
    
    
    #sentences = [w for w in sentences if not w in stop_words_list] 
    #catchphrases = [w for w in catchphrases if not w in stop_words_list]
    
    
    #sentences=[word.lower() for word in sentences if word.isalphanum()]
    #catchphrases=[word.lower() for word in catchphrases if word.isalphanum()]
    
    return sentences,catchphrases

In [12]:
sentences,catchphrases=get_statements(files[0])
start=time.time()
sentences, catch_words = fp.get_dataframe(pd.DataFrame(sentences),pd.DataFrame(catchphrases))
print(sentences,catch_words)
print("time-taken:",time.time()-start)

(        Id     file_id  is_catchword                                words
0       s0  07_209.xml             0  [12194, 12194, 12194, 12194, 12194]
1       s0  07_209.xml             1  [12194, 12194, 12194, 12194, 12194]
2       s0  07_209.xml             0  [12194, 12194, 12194, 12194, 12194]
3       s0  07_209.xml             0  [12194, 12194, 12194, 12194, 12194]
4       s0  07_209.xml             0  [12194, 12194, 12194, 12194, 12194]
5       s0  07_209.xml             0  [12194, 12194, 12194, 12194, 12194]
6       s0  07_209.xml             0  [12194, 12194, 12194, 12194, 12194]
7       s0  07_209.xml             0  [12194, 12194, 12194, 12194, 12194]
8       s0  07_209.xml             0  [12194, 12194, 12194, 12194, 12194]
9       s0  07_209.xml             0  [12194, 12194, 12194, 12194, 12194]
10      s1  07_209.xml             0  [12194, 12194, 12194, 12194, 12194]
11      s1  07_209.xml             0  [12194, 12194, 12194, 12194, 12194]
12      s1  07_209.xml             0 

In [13]:
def prepare_index_matrices(sentence_indexes):
    length=len(sentence_indexes);
    
    sentence_indexes=[word_index['unk'],word_index['unk']]+sentence_indexes+[word_index['unk'],word_index['unk']];    
    out_indices=[]
    for i in range(2,length):
        out_indices.append(sentence_indexes[i-2:i+3]);
        
    return out_indices;

def prepare_label_matrices(labels):
    length=len(labels);
    labels=[0,0]+labels+[0,0];
    out_labels=[];
    
    for i in range(2,length):
        out_labels.append(labels[i]);
        
    return out_labels;
    

In [14]:
def lookup_indexes_labels(sentences,catchphrases):
    
    sentence_indexes=[];
    labels=[];
    
    for word in sentences:
        if word in word_index:
            sentence_indexes.append(word_index[word]);
        else:
            #print("unknown word",word)
            sentence_indexes.append(word_index['unk']);
            
    for word in sentences:
        if word in catchphrases:
            labels.append(1.0);
        else:
            #print("unknown word",word)
            labels.append(0.0);
            
    return sentence_indexes,labels;

In [15]:
def get_file_sentences(file_name):
    temp_data=full_data[(full_data.file_id==file_name) & full_data['Id'].str.startswith(('s'))]
    return list(temp_data.text.values)[:-1]

In [51]:
import time

class CatchPhraseExtractor(object):

    def __init__(self, graph, embedding_array_1):
        self.build_graph(graph, embedding_array_1)

    def build_graph(self, graph, embedding_array_2):
        print("generating graph");

        with graph.as_default():            
            self.phrases_indices = tf.placeholder(tf.int32, shape=[None, 5])
            self.phrase_labels = tf.placeholder(tf.float32, shape=[None, 1])
            
            self.test_inputs_words = tf.placeholder(tf.int32, shape=[None, 5])
            
            
            self.W = tf.Variable(embedding_array_2, dtype=tf.float32)
            #W = tf.Variable(tf.random_uniform([vocabulary_size, embedding_size], -1.0, 1.0))
            phrase_tensor = tf.nn.embedding_lookup(self.W, self.phrases_indices)
            
            #self.sentence_indices = tf.placeholder(tf.int32, shape=[None,1])
            #self.sentence_embeddings = tf.placeholder(tf.float32, shape=[None,200])
            #sentence_tensor = tf.nn.embedding_lookup(self.sentence_embeddings, self.sentence_indices)
            #sentence_tensor=tf.reshape(sentence_tensor, [tf.shape(sentence_tensor)[0], 200])
                                                     
            #self.doc_embedding = tf.placeholder(tf.float32, shape=[1,embedding_size])
            #document_tensor = tf.tile(self.doc_embedding, [tf.shape(self.phrases_indices)[0], 1])
            
            phrase_tensor = tf.reshape(phrase_tensor, [tf.shape(phrase_tensor)[0], embedding_size*5])
            conv_filter = tf.Variable(tf.truncated_normal((embedding_size*5, embedding_size), stddev=0.1))
            word_tensor = tf.nn.relu(tf.matmul(phrase_tensor, conv_filter))
            
            #stacked_tensor = tf.concat(values=[word_tensor, sentence_tensor, document_tensor], axis=1)
            #stacked_tensor = tf.concat(values=[word_tensor, document_tensor], axis=1)
            stacked_tensor = tf.concat(values=[word_tensor], axis=1)
            weights_input = tf.Variable(tf.random_normal((embedding_size*1, 100), stddev=0.2))
            weights_input_2 = tf.Variable(tf.random_normal((100, 1), stddev=0.2))
            
            bias1 = tf.Variable(tf.random_normal((1, 100), stddev=0.2))
            bias2 = tf.Variable(tf.random_normal((1, 1), stddev=0.2))
            
            self.sentence_values=self.MLP(stacked_tensor, weights_input, weights_input_2, bias1, bias2)
            
            #prediction_loss = tf.nn.softmax_cross_entropy_with_logits_v2(labels=self.phrase_labels,logits=self.sentence_values)
            #self.loss = tf.reduce_mean(prediction_loss)
            self.loss=tf.reduce_mean(tf.keras.metrics.mean_squared_error(self.phrase_labels, self.sentence_values))
            #self.loss=tf.convert_to_tensor(tf.reduce_mean(self.sentence_values));
            
            #using the gradient descent optimizer
            optimizer = tf.train.GradientDescentOptimizer(learning_rate)
            grads = optimizer.compute_gradients(self.loss)
            #clipped_grads = [(tf.clip_by_norm(grad, 5), var) for grad, var in grads]
            self.app = optimizer.apply_gradients(grads)
            
            #For test data
            test_embed = tf.nn.embedding_lookup(self.W, self.test_inputs_words)
            test_conv_filter = tf.Variable(tf.truncated_normal((embedding_size*5, embedding_size), stddev=0.1));
            test_word_tensor = tf.nn.relu(tf.matmul(test_embed, test_conv_filter));
            self.test_pred = self.MLP(test_word_tensor, weights_input, weights_input_2, bias1, bias2)
            
            self.init = tf.global_variables_initializer()
            
    def MLP(self, stacked_tensor, weights_input, weights_input_2, bias1, bias2):
        h1 = (tf.matmul(weights_input,stacked_tensor,transpose_a=True,transpose_b=True))
        #h1 = tf.add(h1, bias1)
        h1 = tf.nn.tanh(h1)
        h2 = tf.matmul(weights_input_2,h1,transpose_a=True,transpose_b=False)
        #h2 = tf.add(h2, bias2)
        h2 = tf.nn.sigmoid(h2)
        #h2 = tf.round(tf.transpose(h2))
        return tf.transpose(h2)
        #return tf.cast(tf.to_float(h2>0.5),dtype=tf.float32)

    '''
    def gradient_descent(self,total_loss):
        total_loss=tf.convert_to_tensor(total_loss)
        with tf.GradientTape(persistent=True) as tape:
            variables=tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES)
            print(variables)
        return total_loss, tape.gradient(tf.convert_to_tensor(total_loss), variables)
    '''
    
    def train(self, sess, num_steps):
        self.init.run()
        #print("Initailized")
        start=time.time()
        total_loss=0.0;
        for file in files[:-200]:
            start=time.time()
            sentences, catchphrases=get_statements(file);
            file_data_frame, catch_words = fp.get_dataframe(pd.DataFrame(sentences),pd.DataFrame(catchphrases))
            #print("Time taken:",time.time()-start)
            
            #needs sentences list
            #sentences_list = fp.get_sentences(file) #indexes/order preserved
            #sentence_embeddings = sen2vec(sentences_list)
            #sentence_embeddings = get_sent_embeddings(sentences_list)
            
            #sentence_ids_to_indices = np.array([int(x[1:]) for x in list(file_data_frame['Id'].values)])
            
            #phrase inputs and labels
            phrases_indices = np.array(list(file_data_frame['words'].values))
            phrases_label_matrix = np.array(list(file_data_frame['is_catchword'].values))
            
            
            #doc embedding
            #doc_embedding = d2v_model.docvecs[file]

            
            #doc_embedding = np.reshape(doc_embedding, (1, embedding_size))
            #sentence_ids_to_indices = np.reshape(sentence_ids_to_indices, (sentence_ids_to_indices.shape[0], 1))
            phrases_label_matrix = np.reshape(phrases_label_matrix, (phrases_label_matrix.shape[0], 1))
            
            '''
            print ('*************Doc: ', doc_embedding.shape)
            print ('*************phrases_indices: ', phrases_indices.shape)
            print ('*************sentence_ids_to_indices: ', sentence_ids_to_indices.shape)
            print ('*************phrases_label_matrix: ', phrases_label_matrix.shape)
            print ('*************sentence_embeddings: ', sentence_embeddings.shape)           
            
            feed_dict = {self.doc_embedding: doc_embedding, 
                         self.phrases_indices: phrases_indices, 
                         self.phrase_labels: phrases_label_matrix, 
                         self.sentence_indices:sentence_ids_to_indices, 
                         self.sentence_embeddings: sentence_embeddings}'''
            
            feed_dict = {#self.doc_embedding: doc_embedding, 
                         self.phrases_indices: phrases_indices, 
                         self.phrase_labels: phrases_label_matrix}
            
            #print("sending inputs");
            #loss_val = sess.run([self.loss], feed_dict=feed_dict)  
            _,loss_val,outputs = sess.run([self.app,self.loss,self.sentence_values], feed_dict=feed_dict)

            if num_steps%20 == 0:
                print('Train Finished for ', num_steps, ' loss_val=', loss_val)
            #for i in range(len(outputs)):
            #    print(outputs[i])
       
    def get_phrases(phrase_indices, output):
        catch_phrases = []
        for i, out in enumerate(output):
            if out == 1:
                phrase = ''
                for ind in range(5):
                    phrase += reverseWordIndex[phrase_indices[i][ind]] + " "
                catch_phrases.append(phrase[:-1])
        return catch_phrases
            

    def test(self, sess, file):
        sentences, catchphrases=get_statements(file);
        file_data_frame, catch_words = fp.get_dataframe(pd.DataFrame(sentences),pd.DataFrame(catchphrases))
        
        phrases_indices = np.array(list(file_data_frame['words'].values))
        phrases_label_matrix = np.array(list(file_data_frame['is_catchword'].values))
        phrases_label_matrix = np.reshape(phrases_label_matrix, (phrases_label_matrix.shape[0], 1))
            
        feed_dict = {self.phrases_indices: phrases_indices, 
                     self.phrase_labels: phrases_label_matrix}
        outputs = sess.run([self.test_pred], feed_dict=feed_dict)

        phrase_list = get_phrases(phrase_indices, outputs)
        
        
        
    def load_embeddings(self,embedding_array):
        sess.run(self.embedding_init, feed_dict={self.embedding_placeholder: np.asarray(list(embedding_array.values()))})
        

    

In [42]:
def get_features(inputs):
    out=np.zeros((len(inputs),embedding_size));
    for i in range(len(inputs)):
        try:
            c = w2v[inputs[i]]
        except KeyError:
            c = np.zeros((1,embedding_size));
        out[i]=c;
    
    return np.transpose(out);

In [52]:
graph = tf.Graph()

model = CatchPhraseExtractor(graph, embedding_array)
start=time.time()

#print(fp.get_dataframe(files[0]))
#sentences_list = fp.get_file_sentences(files[0]) #indexes/order preserved
#sentence_embeddings = sen2vec(sentences_list)
end=time.time()
print("time taken:",end-start);

generating graph
('time taken:', 2.5033950805664062e-05)


In [53]:
with tf.Session(graph=graph) as sess:
    for i in range(1):
        model.train(sess, i)
    #optimizer.apply_gradients(zip(grads, model.variables), global_step)
        

('Train Finished for ', 0, ' loss_val=', 0.24552563)
('Train Finished for ', 0, ' loss_val=', 0.21069528)
('Train Finished for ', 0, ' loss_val=', 0.17224753)
('Train Finished for ', 0, ' loss_val=', 0.13307263)
('Train Finished for ', 0, ' loss_val=', 0.10105105)
('Train Finished for ', 0, ' loss_val=', 0.08636232)
('Train Finished for ', 0, ' loss_val=', 0.08922961)
('Train Finished for ', 0, ' loss_val=', 0.07187861)
('Train Finished for ', 0, ' loss_val=', 0.09461574)
('Train Finished for ', 0, ' loss_val=', 0.048303325)
('Train Finished for ', 20, ' loss_val=', 0.25390908)
('Train Finished for ', 20, ' loss_val=', 0.23377506)
('Train Finished for ', 20, ' loss_val=', 0.20994931)
('Train Finished for ', 20, ' loss_val=', 0.1847222)
('Train Finished for ', 20, ' loss_val=', 0.15832828)
('Train Finished for ', 20, ' loss_val=', 0.14013885)
('Train Finished for ', 20, ' loss_val=', 0.13135284)
('Train Finished for ', 20, ' loss_val=', 0.1111029)
('Train Finished for ', 20, ' loss_val=

KeyboardInterrupt: 